In [ ]:
# Update this with your own bucket

bucket_name = 'jsimon-public-us'

In [ ]:
%%sh -s $bucket_name
zip -9 lambda.zip lambda.py
aws s3 cp lambda.zip s3://$1

In [ ]:
import boto3

In [ ]:
!pygmentize trust-policy.json

In [ ]:
iam = boto3.client('iam')

with open('trust-policy.json') as f:
    policy = f.read()

role_name = 'lambda-role-sagemaker-describe-endpoint'

response = iam.create_role(
    RoleName=role_name,
    AssumeRolePolicyDocument=policy,
    Description='Allow function to invoke all SageMaker APIs'
)

role_arn = response['Role']['Arn']
print(role_arn)

In [ ]:
!pygmentize policy.json

In [ ]:
with open('policy.json') as f:
    policy = f.read()
    
policy_name = 'Sagemaker-describe-endpoint'

response = iam.create_policy(
    PolicyName=policy_name,
    PolicyDocument=policy,
    Description='Allow the DescribeEndpoint API'
)

policy_arn = response['Policy']['Arn']
print(policy_arn)

In [ ]:
response = iam.attach_role_policy(
    RoleName=role_name,
    PolicyArn=policy_arn
)

response

In [ ]:
lambda_client = boto3.client('lambda')

response = lambda_client.create_function(
    FunctionName='sagemaker-wait-for-endpoint',
    Role=role_arn,
    Runtime='python3.6',
    Handler='lambda.lambda_handler',
    Code={
        'S3Bucket': bucket_name,
        'S3Key': 'lambda.zip',
    },
    Description='Wait for endpoint to be in service',
    Timeout=900,
    MemorySize=128
)

response

In [ ]:
lambda_client.delete_function(FunctionName='sagemaker-wait-for-endpoint')

In [ ]:
response = iam.detach_role_policy(
    RoleName=role_name,
    PolicyArn=policy_arn
)

response

In [ ]:
response = iam.delete_policy(
    PolicyArn=policy_arn
)

response

In [ ]:
iam.delete_role(RoleName=role_name)